In [66]:
import pandas as pd
import numpy as np

def loadData():
    df = pd.read_csv('~/goinfre/data/train_concat_95_00001.csv')
    y = pd.read_csv('~/Downloads/y.csv', sep=';', usecols=[1])
    #df = pd.read_csv('E:\\Downloads\\train_concat_95_00001.csv')
    #y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1])
    print("Datas loaded")
    return df, y

df, y = loadData()
test = pd.read_csv('~/goinfre/data/test_concat_95_00001.csv')
print(df.shape)

Datas loaded
(26500, 6279)


In [67]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier

def buildforest(train, y):
    forest = RandomForestClassifier(n_estimators=500, oob_score=False, n_jobs=-1, min_samples_split=2, min_samples_leaf=10, random_state=42)
    forest.fit(train, y.values.ravel())
    #print(f'Out-of-bag score estimate: {forest.oob_score_:.3}')
    return forest

%time forest = buildforest(df, y)
prob_pos = forest.predict_proba(test)[:, 1]
#fpr, tpr, _ = roc_curve(y_test, prob_pos)
#roc_auc = auc(fpr, tpr)
#print('ROC AUC: %0.3f' % (roc_auc))

CPU times: user 2min 28s, sys: 1.62 s, total: 2min 29s
Wall time: 47.9 s


In [83]:
from sklearn.externals import joblib

test = pd.DataFrame(columns={'Ids', 'TARGET'})
test['Ids'] = [i for i in range(26500,39750)]
test['TARGET'] = prob_pos
test.to_csv("~/Downloads/submission.csv", sep=';', index=False, header={'Ids', 'TARGET'})

joblib.dump(forest, 'RF.pkl') 

In [5]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(forest, threshold="2.4*mean", prefit=True)
dfmin = model.transform(df)
print(dfmin.shape)
print(df.shape)
dfmin.to_csv('/sgoinfre/goinfre/Perso/pdespres/ensdata/train_importance100.csv', index=False)

(26500, 601)
(26500, 18124)


In [9]:
%time newforest = buildforest(dfmin, y)

Out-of-bag score estimate: 0.683
CPU times: user 29.8 s, sys: 1.69 s, total: 31.5 s
Wall time: 27.7 s


In [6]:
importances = forest.feature_importances_
coltodrop = []
#std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::]

#np.savetxt("/sgoinfre/goinfre/Perso/pdespres/ensdata/features_importances_2000.csv", indices, delimiter=",")
#np.savetxt('E:\\Downloads\\features_importances_2000.csv', indices, delimiter=",")
for f in range(18124):
    coltodrop.append([indices[f],importances[indices[f]]])
    #print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]*1000))
#df.drop(df.columns[coltodrop], axis=1, inplace=True)
#df.shape

test = pd.DataFrame(coltodrop)
test.to_csv('E:\\Downloads\\features_importances_2000.csv', index=False)